In [3]:
from bs4 import BeautifulSoup
# import requests
from requests import get
from requests_html import HTMLSession
import json
import random
import string
import sqlite3
import logging
import pandas as pd
import re


In [10]:
# Input/ Output
db_path = '../Data/test.db'
db_name = 'test'
log_path = '../Logs/test.log'
conn = sqlite3.connect(db_path)
c = conn.cursor()
# c.execute(("""
#        CREATE TABLE IF NOT EXISTS
#            test(
#                NODE TEXT,
#                URL TEXT,
#                REVIEWBODY TEXT,
#                RATING TEXT,
#                REVIEWRATING TEXT,
#                BESTRATING TEXT,
#                WORSTRATING TEXT,
#                PRIMARY KEY (NODE, URL))
#    """))

# Input
# current_file = 'already_links_in/deepfocusreview.txt'
# current_reviews = set(line.strip() for line in open(current_file))

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=log_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [15]:
# print(len(current_reviews))

1129


In [19]:
host = "https://www.cnet.com"
review_count = 0
no_annotation = 0
no_rating = 0
already_count = 0
problem_count = 0

def generateNode(length):
    letters_and_digits = string.ascii_letters +  string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

links = []
for page in range(1, 23):
    url = f'https://www.proporta.com/devices?p={page}/'
    response = get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    lefts = soup.find_all('li', class_='item first')
    for left in lefts:
        links.append(left.a['href'] + '#allReviews')
    rights = soup.find_all('li', class_='item last')
    for right in rights:
        links.append(right.a['href'] + '#allReviews')
for page in range(1, 23):
    url = f'https://www.proporta.com/devices?p={page}/'
    response = get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    lefts = soup.find_all('li', class_='item first')
    for left in lefts:
        links.append(left.a['href'] + '#allReviews')
    rights = soup.find_all('li', class_='item last')
    for right in rights:
        links.append(right.a['href'] + '#allReviews')
print(len(links))

414


In [20]:
print(links[3])

https://www.proporta.com/ted-baker-mirror-case-for-iphone-xr-leyyaa#allReviews


In [41]:
# review_link='https://www.proporta.com/ted-baker-mirror-case-for-iphone-xr-leyyaa#allReviews'
review_link='https://www.proporta.com/ted-baker-cheryia-mirror-folio-case-with-outer-card-slot-for-iphone-8-7-6-hedgerow#allReviews'
response = get(review_link)
soup = BeautifulSoup(response.text, 'lxml')
reviews = soup.find_all('li', itemtype='http://schema.org/Review')
if len(reviews) == 0:
    no_annotation += 1
    pass
else:
    for review in reviews:
        reviewBody = review.find('p', itemprop='reviewBody')
        for word in reviewBody:
            neu = review.find('meta', itemprop='datePublished')
            print(neu)
        posted = review.find('meta', itemprop='datePublished').text
        substring = f"\n {posted}$"
        # print(reviewBody)
        # ratingValue = data['review']['reviewRating']['ratingValue']
        # bestRating = data['review']['reviewRating']['bestRating']
        # worstRating = data['review']['reviewRating']['worstRating']
        # reviewRating = "already included"
        # print(reviewBody)

# //*[@id="reviewContainer"]/li[1]/p/text()[2]


<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-09-27" itemprop="datePublished"/>
<meta content="2019-09-27" itemprop="datePublished"/>
<meta content="2019-09-27" itemprop="datePublished"/>
<meta content="2019-09-27" itemprop="datePublished"/>
<meta content="2019-09-27" itemprop="datePublished"/>
<meta content="2019-08-29" itemprop="datePublished"/>
<meta content="2019-08-29" itemprop="datePublished"/>
<meta content="2019-08-29" itemprop="datePublished"/>
<meta content="2019-08-29" itemprop="datePublished"/>
<meta content="2019-08-29" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" itemprop="datePublished"/>
<meta content="2019-10-09" i

In [ ]:
# data = json.loads("".join(soup.find("script", {"type": "application/ld+json"}).contents),strict=False)
# # print(data)
# try:
#     reviewBody = data['review']['reviewBody']
#     try:
#         ratingValue = data['review']['reviewRating']['ratingValue']
#         bestRating = data['review']['reviewRating']['bestRating']
#         worstRating = data['review']['reviewRating']['worstRating']
#         reviewRating = "already included"
#     except:
#         no_rating += 1
# except:
#     print('no_annotation')

 # response = get(url)
    # soup = BeautifulSoup(response.text, 'lxml')
    # left_products = soup.find_all('section', class_='col-3 searchItem product left')
    # products = soup.find_all('section', class_='col-3 searchItem product')
    # for prod in products:
    #     links.append(prod.a['href'])
    # for prod in left_products:
    #     links.append(prod.a['href'])

# review = soup.find_all('div', itemtype='http://schema.org/Review')
# if len(review) == 0:
#     pass
# else:
#     node = generateNode(31)
#     url = review_link
#

# soup.find_all('div', class_='article-card__content')
# if review_link in current_reviews:
#     print('already in')
#     pass
# else:
#     response = get(review_link)
#     if response.status_code != 200:
#         problem_count += 1
#         # print(review_link)
#     else:
#         review_soup = BeautifulSoup(response.text, 'lxml')
#         # print(review_soup)
#         node = generateNode(31)
#         url = review_link
#         reviewBody = ""
#         for i in review_soup.find_all('div', itemprop='reviewBody'):
#             for j in i.find_all('p'):
#                 reviewBody = reviewBody + j.getText()
#
#         print('node:' + node)
#         print('url: '+ review_link)
#         print('reviewBody: ' + reviewBody)
#         print('worstRating: ' + worstRating)
#         print('bestRating: ' + bestRating)
#         print('ratingValue: ' + ratingValue)
#
#         c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
#         conn.commit()

# print(f"Done {host} - Reviews extracted: " + str(review_count) + ", without Rating: " + str(no_rating) + ", problems: " + str(problem_count))

# host = 'https://www.telegraph.co.uk/'
# url = 'https://www.telegraph.co.uk/films/reviews/page-33/'
# soup = BeautifulSoup(get(url).text, 'lxml')
# movie_container = soup.find_all("div", class_="card__content")
# # print(movie_container)
# for link in movie_container:
#     # print(link)
#     # print(link.a['href'])
#     review_link = host + link.a['href']
#     # if not "movie-reviews" in review_link:
#     #     pass
#     # else:
#     print(review_link)

#     # print(link.get('href'))
#     if review_link is not None:
#         if review_link.endswith('html'):
#             review_soup = BeautifulSoup(get(review_link).text, 'lxml')
#             review_html = review_soup.find("div", itemtype="http://schema.org/Review")
#             # print(review_html)
#             reviewBody = ""
#             if review_html is None:
#                 print("Kein Review Annotation")
#                 no_annotation += 1
#             else:
#                 reviewBody_html = review_html.find_all("div", class_="reviewtext")
#                 reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")
#                 for item in reviewBody_html:
#                     reviewBody += item.p.text
#



# Snippets
#
# for character in string.ascii_lowercase + '0':
#     url = f'https://deepfocusreview.com/reviews/?id={character}'
#     response = get(url)
#     soup = BeautifulSoup(response.text, 'lxml')
#     for entry in soup.find_all('ol', id='reviewalpha-list'):
#         for link in entry.find_all('li'):
#             # print(link.a['href'])
#             review_count += 1

# def generateNode(length):
#     letters_and_digits = string.ascii_letters +  string.digits
#     result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
#     node = "_:znode" + result_str
#     return node

    # node = generateNode(31)

# for page in range(1, 414):
#     url = f'https://www.rollingstone.com/movies/movie-reviews/page/{page}/'

# review_soup = BeautifulSoup(get(review_link).text, 'lxml')
# review_html = review_soup.find("div", itemtype="http://schema.org/Review")
# review_html = review_soup.find(id="pmc-movie-review-snippet")

# response = get(review_link, headers={'User-Agent': 'Custom'})
# if response.status_code == 404:
#     fail_lst.append(review_link)

# for item in review_html:
#     jsonExtract = str(item).split('\n')[2].replace(";", "")
#     data = json.loads("".join(jsonExtract))

    # reviewBody = data['description']
    # ratingValue = str(data['reviewRating']['ratingValue'])
    # bestRating = str(data['reviewRating']['bestRating'])
    # worstRating = str(data['reviewRating']['worstRating'])
    # reviewRating = "already included"

    # reviewBody = soup.find_all("div", itemprop="reviewBody")[0].text
    # ratingValue = soup.find_all("span", itemprop="ratingValue")[0].text
    # worstRating = soup.find_all("meta", itemprop="worstRating")[0]['content']
    # bestRating =soup.find_all("meta", itemprop="bestRating")[0]['content']

    # ratingValue = review_soup.find('meta', itemprop='ratingValue')['content']
    # worstRating = review_soup.find('meta', itemprop='worstRating')['content']
    # bestRating = review_soup.find('meta', itemprop='bestRating')['content']
    # reviewRating = "already included"

# data = json.loads("".join(review_soup.find("script", type="application/ld+json").contents))
# data = json.loads("".join(review_soup.find("script", {"type": "application/ld+json"}).contents),strict=False)

# reviewBody_html = review_html.find_all("div", class_="reviewtext")

# reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")

# reviewBody += item.p.text

# worstRating = reviewRating.find("meta", itemprop="worstRating")["content"]
# bestRating = reviewRating.find("meta", itemprop="bestRating")["content"]
# ratingValue = reviewRating.find("meta", itemprop="ratingValue")["content"]

# print('node:' + node)
# print('url: '+ review_link)
# print('reviewBody: ' + reviewBody)
# print('worstRating: ' + worstRating)
# print('bestRating: ' + bestRating)
# print('ratingValue: ' + ratingValue)

# c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
# conn.commit()

# soup = BeautifulSoup(get(url).text, 'html.parser')
# movie_container = soup.find_all("div", class_="entry post clearfix")
# for link in movie_container[0].find_all("a"):
#     review_link = link.get('href')
#     print(link.get('href'))
#     if review_link is not None:
#         if review_link.endswith('html'):
#             if review_link in current_reviews:
#                 pass
#             else:
#                 print(review_link)
#                 manche haben schema.org/Review nicht drin
#                 review_count += 1
#         if review_link.endswith('html') & review_link not in current_reviews:
#             print(review_link)
#         else:
#             pass
#                 #nicht alle Einträge sind vollständig (fehlende Bewertung)
#                 # review_count += 1
#     else:
#         pass